In [3]:
data_dir = '/home/ubuntu/data/iceberg'

import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, Cropping2D
from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.layers.merge import Add, Concatenate
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
import cv2
import keras
import os
from keras.callbacks import TensorBoard

from keras import __version__
print(__version__)


2.1.1


In [4]:
np.random.seed(1234)

def get_scaled_imgs(df):
    imgs = []

    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)

        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

#         imgs.append(np.dstack((band_1, band_2, band_3)))
        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [5]:
df_train = pd.read_json(os.path.join(data_dir, 'train.json'))
Xtrain = get_scaled_imgs(df_train)
Ytrain = np.array(df_train['is_iceberg'])

df_train.inc_angle = df_train.inc_angle.replace('na',0)
idx_tr = np.where(df_train.inc_angle>0)

Ytrain = Ytrain[idx_tr[0]]
Xtrain = Xtrain[idx_tr[0],...]
Xinc = df_train.inc_angle[idx_tr[0]]

In [6]:
Xtrain[1,:,:,1].max()

0.72976856454753891

In [7]:
Xtrain = get_more_images(Xtrain)
Xinc = np.concatenate((Xinc,Xinc,Xinc))
Ytrain = np.concatenate((Ytrain,Ytrain,Ytrain))

In [13]:
batch_size = 256
        
tbCallBack = TensorBoard(log_dir='/home/ubuntu/data/tensorboardlogs/', histogram_freq=0, write_graph=True, write_images=True)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, verbose=1, epsilon=1e-4, mode='min')

In [111]:
p = 0.2
input_shape = (75, 75, 3)
num_classes = 2

classifier_input = Input(shape=input_shape)
inc_angle_input = Input(shape=(1,))

# CNN 1
x = Conv2D(64, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 2
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Dropout(p)(x)

# CNN 3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 3
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# x = BatchNormalization(axis=-1)(x)

# CNN 4
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
# x = BatchNormalization(axis=-1)(x)
x = Dropout(p)(x)

x = Flatten()(x)

# x = GlobalAveragePooling2D()(x)
m = Concatenate()([inc_angle_input, x])
m = Dense(512, activation='relu')(m)
m = Dense(256, activation='relu')(m)
out = Dense(2, activation='sigmoid')(m)
# out = Activation('softmax')(m)

# optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = Adam(lr=0.0015, decay=0.0)
model = Model(inputs=[classifier_input, inc_angle_input], outputs=out)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [113]:
model.fit([Xtrain, Xinc], to_categorical(Ytrain), batch_size=batch_size, epochs=50, verbose=1,shuffle=True,
          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack], validation_split=0.25)

Train on 3309 samples, validate on 1104 samples
Epoch 1/50
3309/3309 [==============================] - 6s 2ms/step - loss: 0.9114 - acc: 0.4995 - val_loss: 0.8247 - val_acc: 0.4937
Epoch 2/50
3309/3309 [==============================] - 6s 2ms/step - loss: 0.7453 - acc: 0.4977 - val_loss: 0.6931 - val_acc: 0.5063
Epoch 3/50
3309/3309 [==============================] - 6s 2ms/step - loss: 0.6974 - acc: 0.4992 - val_loss: 0.6928 - val_acc: 0.5063
Epoch 4/50
3309/3309 [==============================] - 6s 2ms/step - loss: 0.6879 - acc: 0.5128 - val_loss: 0.6454 - val_acc: 0.6431
Epoch 5/50
3309/3309 [==============================] - 6s 2ms/step - loss: 0.6419 - acc: 0.6233 - val_loss: 0.5854 - val_acc: 0.6594
Epoch 6/50
3309/3309 [==============================] - 6s 2ms/step - loss: 0.5927 - acc: 0.6769 - val_loss: 0.6633 - val_acc: 0.6427
Epoch 7/50
3309/3309 [==============================] - 6s 2ms/step - loss: 0.5230 - acc: 0.7403 - val_loss: 0.4789 - val_acc: 0.7813
Epoch 8/50
330

In [18]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 75, 75, 32)   896         input_11[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_12 (MaxPooling2D) (None, 37, 37, 32)   0           conv2d_26[0][0]                  
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, 37, 37, 64)   18496       max_pooling2d_12[0][0]           
__________________________________________________________________________________________________
max_poolin

In [89]:
# model.load_weights(filepath = '.mdl_wts.hdf5')
score = model.evaluate([Xtrain,Xinc], to_categorical(Ytrain), verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

df_test = pd.read_json(os.path.join(data_dir, 'test.json'))
df_test.inc_angle = df_test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(df_test))
Xtest_inc = df_test.inc_angle
pred_test = model.predict([Xtest,Xtest_inc])


4413/4413 [==============================] - 3s 654us/step
Train score: 0.082554457009
Train accuracy: 0.972014502484


In [107]:
pred_test[1,1]

0.9525063

In [106]:
# submit_nums = np.clip(pred_test[:,1], 0.1, 0.9)
submit_nums[1]

0.89999998

In [109]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test[:,1].reshape((pred_test[:,1].shape[0]))})
print(submission.head(10))

submission.to_csv('cnn_train_' + str(np.around(score[1], decimals=2))  + '.csv', index=False)

         id  is_iceberg
0  5941774d    0.001687
1  4023181e    0.952506
2  b20200e4    0.778264
3  e7f018bb    0.997760
4  4371c8c3    0.273599
5  a8d9b1fd    0.829154
6  29e7727e    0.055702
7  92a51ffb    0.998996
8  c769ac97    0.000014
9  aee0547d    0.000027


In [92]:
df_test.shape

(8424, 4)

In [114]:
model.evaluate([Xtrain,Xinc], to_categorical(Ytrain), verbose=1)

4413/4413 [==============================] - 3s 600us/step


[0.13649047835023664, 0.95014729196998826]